In [1]:
from joblib import load
from UTILS import *
from Split import test_df as data
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [3]:
preprocess_dict=load("preprocessing_option1.joblib")
# Use the loaded dictionary to transform names in new data
data["state_mean"] = data["State"].map(preprocess_dict["state_dict"])
data["sub_cat_encoded"] = data["Sub Category"].map(preprocess_dict["sub_cat_dict"])

In [4]:
model=load("rf_option1.joblib")

In [5]:
one_hot_df=data[["Main Category","Region"]]#
encoder=model["onehot_encoder"]
final_cat_df=pd.DataFrame(encoder.transform(one_hot_df),columns=encoder.get_feature_names_out(one_hot_df.columns),index=data.index)

In [6]:
final_df_num=data[preprocess_dict["num_columns"]]


<h2>Loading Model</h2>

In [7]:
final_df=pd.concat([final_df_num,final_cat_df],axis=1)
final_df_fill=fill_missing(final_df.drop(columns="Profit"),preprocess_dict["missing_values"])
final_df_norm=pd.DataFrame(model["scaler"].transform(final_df_fill),columns=final_df_fill.columns)

In [8]:
preprocess_dict["missing_values"]

array([2.28211970e+02, 3.76422764e+00, 1.55884928e-01, 3.01305338e+01,
       1.00000000e+01, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00])

In [9]:
final_df.columns

Index(['Sales', 'Quantity', 'Discount', 'Profit', 'state_mean',
       'sub_cat_encoded', 'Main Category_Furniture',
       'Main Category_Office Supplies', 'Main Category_Technology',
       'Region_Central', 'Region_East', 'Region_South', 'Region_West'],
      dtype='object')

In [10]:
X=final_df_norm
y=final_df["Profit"]

<h3>Random Forest</h2>

In [12]:
y_pred = model["model"]["rf"].predict(X)
r2_op1 = r2_score(y, y_pred)
mse_op1 = mean_squared_error(y, y_pred)

print("R^2 Score: ", r2_op1)
print("Mean Squared Error (MSE): ", mse_op1)

R^2 Score:  0.9142622347977666
Mean Squared Error (MSE):  3135.7370572682144


<h3>Polynomial Regression</h3>

In [13]:
x_poly=PolynomialFeatures(degree=model["model"]["plreg"][1]).fit_transform(X)
y_pred = model["model"]["plreg"][0].predict(x_poly)
r2_op1 = r2_score(y, y_pred)
mse_op1 = mean_squared_error(y, y_pred)

print("R^2 Score: ", r2_op1)
print("Mean Squared Error (MSE): ", mse_op1)

R^2 Score:  0.811782151721069
Mean Squared Error (MSE):  6883.8006250267335


<h3>Gradient Boosting Regressor</h3>

In [14]:
y_pred = model["model"]["gbr"].predict(X)
r2_op1 = r2_score(y, y_pred)
mse_op1 = mean_squared_error(y, y_pred)
print("R^2 Score: ", r2_op1)
print("Mean Squared Error (MSE): ", mse_op1)


R^2 Score:  0.9214642288952606
Mean Squared Error (MSE):  2872.334346403634
